In [27]:
import pandas as pd

In [28]:
filepath = 'matthew/Results/JobNoOccupyresults.csv'
df = pd.read_csv(filepath)
df

,utility,fairness,min_utility,objective,variance,greedy,training,reallocate,central_rewards,simple_obs,...,model_loc,u_model_loc,f_model_loc,split,learn_fairness,learn_utility,multi_head,phased_training,phase_length,observation_space
0,98.98,-74.234700,0.00,98.980000,1837.12125,False,True,False,False,True,...,NaN,NaN,NaN,False,True,True,False,False,200,"['relative_job', 'grid', 'relative_util']"
1,98.88,-74.159700,0.00,98.880000,1833.39750,False,True,False,False,False,...,NaN,NaN,NaN,False,True,True,False,False,200,"['loc', 'util', 'relative_job', 'relative_util..."
2,98.44,-73.829700,0.00,98.440000,1817.25000,False,True,False,False,True,...,NaN,NaN,NaN,True,True,True,True,False,200,"['relative_job', 'grid', 'relative_util']"
3,96.56,-39.639719,0.00,96.560000,955.41250,False,True,False,False,False,...,NaN,NaN,NaN,True,True,True,True,False,200,"['loc', 'util', 'relative_job', 'relative_util..."
4,98.46,-55.320869,0.00,30.277938,1363.64125,False,True,False,False,True,...,NaN,NaN,NaN,False,True,True,False,False,200,"['relative_job', 'grid', 'relative_util']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,20.94,-1.718707,2.50,7.227500,6.85625,False,True,False,False,True,...,NaN,Models/JobNoOccupy/Simple/split_diff/Joint/0.0...,NaN,True,True,False,False,False,200,"['relative_job', 'grid', 'relative_util']"
57,73.44,-10.099242,3.94,-324.035000,198.73750,False,True,False,False,False,...,NaN,Models/JobNoOccupy/split_diff/Joint/0.0/170733...,NaN,True,True,False,False,False,200,"['loc', 'util', 'relative_job', 'relative_util..."
58,63.32,-0.353685,13.60,45.345000,3.59500,False,True,False,False,True,...,NaN,Models/JobNoOccupy/Simple/split_diff/Joint/0.0...,NaN,True,True,False,False,False,200,"['relative_job', 'grid', 'relative_util']"
59,7.52,-4.043148,0.00,-334.005000,68.30500,False,True,False,False,False,...,NaN,Models/JobNoOccupy/split_diff/Joint/0.0/170733...,NaN,True,True,False,False,False,200,"['loc', 'util', 'relative_job', 'relative_util..."


In [29]:
splitsimple = df[(df.split==True) & (df.simple_obs==True) & (df.learn_utility==True)]
splitcomplex = df[(df.split==True) & (df.simple_obs==False) & (df.learn_utility==True)]
simplenoutil = df[(df.split==True) & (df.simple_obs==True) & (df.learn_utility==False)]
complexnoutil = df[(df.split==True) & (df.simple_obs==False) & (df.learn_utility==False)]
jointsimple = df[(df.split==False) & (df.simple_obs==True)]
jointcomplex = df[(df.split==False) & (df.simple_obs==False)]


In [30]:
import plotly.express as px
# Sort the dataframes by learning_beta
splitsimple_sorted = splitsimple.sort_values('learning_beta')
splitcomplex_sorted = splitcomplex.sort_values('learning_beta')
simplenoutil_sorted = simplenoutil.sort_values('learning_beta')
complexnoutil_sorted = complexnoutil.sort_values('learning_beta')
jointsimple_sorted = jointsimple.sort_values('learning_beta')
jointcomplex_sorted = jointcomplex.sort_values('learning_beta')


# # Create scatter plots with connected lines
# fig = px.line(splitsimple_sorted, x='utility', y='fairness', title='Split Simple', text='learning_beta')
# fig.show()

# fig = px.line(splitcomplex_sorted, x='utility', y='fairness', title='Split Complex', text='learning_beta')
# fig.show()

# fig = px.line(jointsimple_sorted, x='utility', y='fairness', title='Joint Simple', text='learning_beta')
# fig.show()

# fig = px.line(jointcomplex_sorted, x='utility', y='fairness', title='Joint Complex', text='learning_beta')
# fig.show()

splitsimple_sorted['df_name'] = 'Split Simple'
splitcomplex_sorted['df_name'] = 'Split Complex'
simplenoutil_sorted['df_name'] = 'Simple No Util'
complexnoutil_sorted['df_name'] = 'Complex No Util'
jointsimple_sorted['df_name'] = 'Joint Simple'
jointcomplex_sorted['df_name'] = 'Joint Complex'

# Combine all dataframes into one
combined_df = pd.concat([splitsimple_sorted, splitcomplex_sorted, simplenoutil_sorted, complexnoutil_sorted, jointsimple_sorted, jointcomplex_sorted])

# Plot the lines on a single plot with df name as color
fig = px.line(combined_df, x='utility', y='fairness', color='df_name', title='Combined Dataframes', text='learning_beta', markers=True)
fig.update_layout(width=800, height=600)
fig.show()

In [32]:
# Optimal point is the point with the highest fairness and the highest utility. Fpor each df, extract the pareto frontier
def find_pareto_frontier(df):
    df = df.sort_values('fairness', ascending=False)
    pareto_frontier = [df.iloc[0]]
    for i in range(1, len(df)):
        if df.iloc[i].utility > pareto_frontier[-1].utility:
            pareto_frontier.append(df.iloc[i])
    return pd.DataFrame(pareto_frontier)

pareto_simplesplit = find_pareto_frontier(splitsimple_sorted)
pareto_complexsplit = find_pareto_frontier(splitcomplex_sorted)
pareto_simplenoutil = find_pareto_frontier(simplenoutil_sorted)
pareto_complexnoutil = find_pareto_frontier(complexnoutil_sorted)
pareto_jointsimple = find_pareto_frontier(jointsimple_sorted)
pareto_jointcomplex = find_pareto_frontier(jointcomplex_sorted)

pareto_simplesplit['df_name'] = 'Split Simple'
pareto_complexsplit['df_name'] = 'Split Complex'
pareto_simplenoutil['df_name'] = 'Simple No Util'
pareto_complexnoutil['df_name'] = 'Complex No Util'
pareto_jointsimple['df_name'] = 'Joint Simple'
pareto_jointcomplex['df_name'] = 'Joint Complex'

pareto_combined = pd.concat([pareto_simplesplit, pareto_complexsplit, pareto_simplenoutil, pareto_complexnoutil, pareto_jointsimple, pareto_jointcomplex])
pareto_combined = pd.concat([pareto_simplesplit, pareto_simplenoutil, pareto_jointsimple])

fig = px.line(pareto_combined, x='utility', y='fairness', color='df_name', title='Pareto Frontiers')
fig.update_layout(width=800, height=600)
fig.show()

In [20]:
jointsimple

,utility,fairness,min_utility,objective,variance,greedy,training,reallocate,central_rewards,simple_obs,...,model_loc,u_model_loc,f_model_loc,split,learn_fairness,learn_utility,multi_head,phased_training,phase_length,observation_space
0,98.98,-74.234700,0.00,98.980000,1837.12125,False,True,False,False,True,...,NaN,NaN,NaN,False,True,True,False,False,200,"['relative_job', 'grid', 'relative_util']"
4,98.46,-55.320869,0.00,30.277938,1363.64125,False,True,False,False,True,...,NaN,NaN,NaN,False,True,True,False,False,200,"['relative_job', 'grid', 'relative_util']"
7,97.88,-33.470002,0.00,15.719750,821.60250,False,True,False,False,True,...,NaN,NaN,NaN,False,True,True,False,False,200,"['relative_job', 'grid', 'relative_util']"
10,95.94,-6.969574,8.66,62.441250,167.49375,False,True,False,False,True,...,NaN,NaN,NaN,False,True,True,False,False,200,"['relative_job', 'grid', 'relative_util']"
14,92.38,-1.370684,15.82,76.558125,31.64375,False,True,False,False,True,...,NaN,NaN,NaN,False,True,True,False,False,200,"['relative_job', 'grid', 'relative_util']"
17,86.62,-0.682954,17.00,72.913750,13.70625,False,True,False,False,True,...,NaN,NaN,NaN,False,True,True,False,False,200,"['relative_job', 'grid', 'relative_util']"
20,56.08,-0.639576,11.20,44.750000,5.66500,False,True,False,False,True,...,NaN,NaN,NaN,False,True,True,False,False,200,"['relative_job', 'grid', 'relative_util']"
24,30.66,-0.996124,5.64,12.841250,3.56375,False,True,False,False,True,...,NaN,NaN,NaN,False,True,True,False,False,200,"['relative_job', 'grid', 'relative_util']"
27,49.86,-4.653517,8.04,-918.502500,96.83625,False,True,False,False,True,...,NaN,NaN,NaN,False,True,True,False,False,200,"['relative_job', 'grid', 'relative_util']"
31,54.18,-0.441436,11.54,-49.845000,5.20125,False,True,False,False,True,...,NaN,NaN,NaN,False,True,True,False,False,200,"['relative_job', 'grid', 'relative_util']"
